# Encode Fingerprints

Imports and Constants

**Part 1:** Encoding




This code encodes each fingerprint into a .csv for easy access, as well as the name and image file.




In [17]:
%matplotlib inline
import sys
import os
sys.path.append(os.path.abspath('..'))  

import os, io, base64
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from fastai.learner import load_learner
from itertools import product
import umap

## Part 1: Encoding

In [ ]:
class FlatImageDataset(Dataset):
    def __init__(self, folder, exts=("png", "jpg", "jpeg"), transform=None):
        self.paths = []
        for ext in exts:
            self.paths += sorted(Path(folder).glob(f"*.{ext}"))
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        img = Image.open(p).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(p)

def export_fingerprints_and_thumbnails(model_path, image_dir_A, hover_image_dir_B, output_csv, output_csv_umap):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learn = load_learner(model_path, cpu=(device.type == "cpu"))
    learn.model.to(device).eval()

    # Dataset from Folder A (used for encoding)
    tfm = T.Compose([T.Resize(224), T.ToTensor()])
    ds = FlatImageDataset(image_dir_A, transform=tfm)
    dl = DataLoader(ds, batch_size=64, shuffle=False)

    filenames = []
    latents = []

    with torch.no_grad():
        for xb, names in dl:
            xb = xb.to(device)
            z = learn.model.encoder(xb).flatten(1).cpu().numpy()
            latents.append(z)
            filenames.extend(names)

    latents = np.concatenate(latents)

    # Construct DataFrame
    df = pd.DataFrame(latents, columns=[f"z{i}" for i in range(latents.shape[1])])
    df.insert(0, "filename", filenames)

    df.to_csv(output_csv, index=False)
    print(f"[✓] Exported: {output_csv}")

    reducer = umap.UMAP(
        n_neighbors=10,
        min_dist=0.1,
        metric='minkowski',
        random_state=42
    )

    embedding = reducer.fit_transform(latents)
    df = pd.DataFrame(embedding, columns=[f"z{i}" for i in range(embedding.shape[1])])
    df.insert(0, "filename", filenames)

    df.to_csv(output_csv_umap, index=False)
    print(f"[✓] Exported: {output_csv_umap}")




In [ ]:
resnet_version_param = [18, 50]
latent_space_dim_param = [1, 2, 3]
window_size_param = [1, 2, 3]
training_steps_param = [30, 60, 90, 120]
train_test_split_param = [0, 0.1, 0.5]
line_width_param = [1]

parameters = product(resnet_version_param, latent_space_dim_param, window_size_param, 
                     training_steps_param, train_test_split_param, line_width_param)


In [26]:
for i, (resnet_version, latent_space_dim, window_size, training_steps, train_test_split, line_width) in enumerate(parameters):
    model_name = f"resnet{resnet_version}_latent{latent_space_dim}_window{window_size}eV_steps{training_steps}_split{train_test_split * 10}_width{line_width}_"
    model_path = Path(f"../Model_Generation/models/{model_name}.pkl")
    image_dir_A = f"../BandData/Images/bands_{10 * line_width}width_{window_size}eV_ml/class"
    hover_image_dir_B = f"../BandData/Images/bands_{10 * line_width}width_{window_size}eV/class"
    output_csv = Path(f"./fingerprints/{model_name}.csv")
    output_csv_umap = Path(f"./fingerprints/{model_name}embedding.csv")


    export_fingerprints_and_thumbnails(model_path, image_dir_A, hover_image_dir_B, output_csv, output_csv_umap)


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\fastai\learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


[✓] Exported: fingerprints\resnet18_latent1_window1eV_steps30_split0_width1_.csv


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[✓] Exported: fingerprints\resnet18_latent1_window1eV_steps30_split0_width1_embedding.csv


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\fastai\learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


...

[✓] Exported: fingerprints\resnet18_latent3_window3eV_steps120_split1.0_width1_embedding.csv


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\fastai\learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


[✓] Exported: fingerprints\resnet18_latent3_window3eV_steps120_split5.0_width1_.csv


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[✓] Exported: fingerprints\resnet18_latent3_window3eV_steps120_split5.0_width1_embedding.csv
